In [ ]:
import os
import glob
import pandas as pd
import re
import pprint

## Present a menu of the files with "Survey Student Analysis Report" in the title

In [ ]:
# This is the path to the folder with the data files
os.chdir("/Users/briandomino/Downloads")

extension = 'csv'
all_filenames = [i for i in glob.glob('*Survey Student Analysis Report*.{}'.format(extension))]

In [ ]:
option_number = 1
for filename in all_filenames:
    print(f'[{option_number}] {filename}')
    option_number += 1
print("\n Which file would you like to import, clean, and rename?")
selection = 0

while selection == 0:
    selection = int(input())
    if selection > len(all_filenames):
        print("That's not an option. Please try again.")
        selection = 0

## Create new name for the file before cleaning

In [ ]:
raw_data = pd.read_csv(all_filenames[selection-1])

In [ ]:
# find the most recent date, and take only the year-month-day

survey_date = raw_data['submitted'].max().split(' ')[0]

In [ ]:
# find the section(s)

sections = raw_data['section'].drop_duplicates().reset_index(drop=True)

In [ ]:
all_classes = sections[0]
counter = 1
number_of_sections = len(sections)
while number_of_sections > counter:
        all_classes = all_classes + ', ' + sections[counter]
        counter += 1

In [ ]:
new_file_name = all_classes + " Survey Student Analysis Report " + survey_date + "." + extension

## Clean the file

In [ ]:
# Remove the irrelevant or uninteresting columns

raw_data.drop(columns=['section_id', 'submitted', 'section_sis_id', 'attempt', 'n correct',
                       'n incorrect', 'score'], inplace = True)

In [ ]:
# Since this is an ungraded survey, the scores are meaningless, so we'll drop them

raw_data.drop(list(raw_data.filter(regex = '0.0')), axis = 1, inplace = True)

In [ ]:
# Remove the question numbers from the column names

raw_data.columns = raw_data.columns.str.replace(r'^\d{7}: ','')

In [ ]:
# Remove visible codes for non-breaking spaces and blank lines

raw_data = raw_data.replace({'\xa0':''}, regex=True)
raw_data = raw_data.replace({'\n':''}, regex=True)

### Save cleaned file

In [ ]:
raw_data.to_csv(new_file_name)
print("Data saved to: " + new_file_name)

## Print data from selected column(s)

* This is useful if the survey includes open-ended questions and you want to copy and paste them into another location.
* Since Canvas presents an analysis of the fixed-response questions, and I don't see a way to determine what kind of question each one is, I'm not going to analyze that data

In [ ]:
# Remove the section column since we won't want to copy and paste that
raw_data.drop(columns='section', inplace = True)

In [ ]:
column_names = list(raw_data.columns.values) 
option_number = 0

In [ ]:
print('[0] Finished printing to screen')
selection = 1
while selection < len(column_names):
    print(f'[{selection}] {column_names[selection]}')
    selection += 1

print('\n Which column would you like printed to the screen?')

In [ ]:
selection = -1
while selection == -1:
    try:
        selection = int(input())
        if selection == 0:
            print("Goodbye")
            break
        else:
            print('-----------------------------------------------------')
            pprint.pprint(raw_data[column_names[selection]].unique())
            selection = -1
    except (IndexError, ValueError):
            print("That's not an option. Please try again.")
            selection = -1
